# Texture Synthesis
__Author__ : Mohammad Rouintan , 400222042

__Course__ : Undergraduate Digital Image Processing Course

In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [14]:
def compute_error_surface(block1, block2, overlap, axis=0):
    error_surface = np.sum((block1[:, :overlap] - block2[:, :overlap])**2, axis=2)
    if axis:
        error_surface = error_surface.T
    return error_surface

def find_min_cut(error_surface):
    cost_matrix = np.zeros_like(error_surface)
    cost_matrix[0] = error_surface[0]
    for i in range(1, error_surface.shape[0]):
        for j in range(error_surface.shape[1]):
            min_cost = cost_matrix[i-1, j]
            if j > 0:
                min_cost = min(min_cost, cost_matrix[i-1, j-1])
            if j < error_surface.shape[1] - 1:
                min_cost = min(min_cost, cost_matrix[i-1, j+1])
            cost_matrix[i, j] = error_surface[i, j] + min_cost
    min_cut = np.zeros((error_surface.shape[0]), dtype=np.int8)
    min_cut[-1] = np.argmin(cost_matrix[-1])
    for i in range(error_surface.shape[0]-2, -1, -1):
        j = min_cut[i+1]
        min_j = max(0, j - 1)
        max_j = min(j + 1, error_surface.shape[1] - 1)
        min_cut[i] = j + np.argmin(cost_matrix[i, min_j: max_j+1]) - 1
    return min_cut

def combine_blocks(block1, block2, min_cut, overlap, axis=0):
    if axis:
        block1, block2, min_cut = block1.T, block2.T, min_cut.T
    combined = np.zeros_like(block1)
    for i in range(block1.shape[0]):
        for j in range(block1.shape[1]):
            if j <= min_cut[i]:
                combined[i, j] = block1[i, j]
            else:
                combined[i, j] = block2[i, j]
    if axis:
        combined = combined.T
    return combined

def synthesize_texture(texture, output_size, block_size, overlap):
    synthesized = np.zeros((output_size, output_size, 3), dtype=np.uint8)
    for i in range(0, output_size, block_size - overlap):
        for j in range(0, output_size, block_size - overlap):
            x = np.random.randint(0, texture.shape[0] - block_size)
            y = np.random.randint(0, texture.shape[1] - block_size)
            block = texture[x:x+block_size, y:y+block_size]
            print(block.shape)
            if i > 0:
                error_surface = compute_error_surface(block, synthesized[i-overlap:i, j:j+block_size], overlap, axis=0)
                min_cut = find_min_cut(error_surface)
                block[:overlap] = combine_blocks(block[:overlap], synthesized[i-overlap:i, j:j+block_size], min_cut, overlap, axis=0)
                print(block.shape)
            if j > 0:
                error_surface = compute_error_surface(block, synthesized[i:i+block_size, j-overlap:j], overlap, axis=1)
                min_cut = find_min_cut(error_surface)
                block[:, :overlap] = combine_blocks(block[:, :overlap], synthesized[i:i+block_size, j-overlap:j], min_cut, overlap, axis=1)
                print(block.shape)
            print(block)
            print(block.shape)
            print(synthesized.shape)
            print(synthesized[i:i+block_size, j:j+block_size].shape)
            synthesized[i:i+block_size, j:j+block_size] = block

    return synthesized

def main():
    texture = cv2.imread('Images/texture1.png')
    output_size = 512
    block_size = 64
    overlap = 16

    synthesized = synthesize_texture(texture, output_size, block_size, overlap)
    plt.imshow(synthesized, cmap='gray')
    plt.show()
    # cv2.imwrite('synthesized_texture.jpg', synthesized)

if __name__ == '__main__':
    main()

(64, 64, 3)
[[[ 27  24 239]
  [ 23  22 232]
  [ 68  68 250]
  ...
  [ 71 202 200]
  [ 45 197 192]
  [ 40 196 189]]

 [[ 17  13 226]
  [ 18  15 224]
  [ 60  58 250]
  ...
  [ 75 202 201]
  [ 64 211 206]
  [ 58 209 202]]

 [[ 15   6 214]
  [ 34  25 231]
  [ 61  53 252]
  ...
  [ 33 142 142]
  [ 51 173 170]
  [ 71 207 200]]

 ...

 [[ 63  94  94]
  [ 32  61  46]
  [ 54  91  68]
  ...
  [ 34  31 117]
  [ 37  39 123]
  [ 39  46 124]]

 [[ 51  82  82]
  [ 15  43  28]
  [ 15  49  26]
  ...
  [ 44  41 128]
  [ 41  45 129]
  [ 41  50 128]]

 [[ 35  66  66]
  [ 23  49  33]
  [ 18  44  21]
  ...
  [ 39  39 129]
  [ 35  43 127]
  [ 33  46 126]]]
(64, 64, 3)
(512, 512, 3)
(64, 64, 3)
(64, 64, 3)
(64, 64, 3)
[[[ 23  23 240]
  [ 22  23 242]
  [ 22  23 242]
  ...
  [ 11  36  46]
  [  5  28  50]
  [  2  29  55]]

 [[ 25  25 245]
  [ 26  26 248]
  [ 28  28 250]
  ...
  [  1  19  32]
  [  7  32  56]
  [  7  34  65]]

 [[  9   6 236]
  [ 11   7 241]
  [ 17  12 248]
  ...
  [ 43  19  49]
  [ 44  18  73]
  

ValueError: could not broadcast input array from shape (64,64,3) into shape (64,32,3)